In [1]:
import pandas as pd
import datetime
from config import getClient
import numpy as np

In [10]:
datalib = getClient("OutFront")

In [65]:
cols = ["panel", "date", "hour", "observed", "population"]
df = pd.read_csv(datalib+"panel_counts.csv", header=0,names=cols)

In [66]:
start = df["date"].min()
end   = df["date"].max()
count = len(df["panel"].unique())
print("Full dataset runs from {} - {}".format(start, end))
print("Number of rows: {:,.0f}".format(df.shape[0]))
print("Number of unique panels: {}".format(count))

Full dataset runs from 20170525 - 20180710
Number of rows: 863,238
Number of unique panels: 100


In [67]:
df  = df.set_index("panel")

# Convert their date format to a standard date
df["date"] = pd.to_datetime(df["date"], format='%Y%m%d')

df  = df.sort_values(["date", "hour"])

In [69]:
# Analyze the fill rate and remove any panels with low rate
grp = df.groupby(level=0)
l = []
count = 0
before = df.shape[0]

for panel, val in grp:
    dates = val["date"]
    diff  = (dates.max() - dates.min()).days
    durations = diff*24
    numPoints = len(dates)
    fillRate = numPoints/durations
    if fillRate < .92:
        #print("{:<15}{:.0%}".format(panel, fillRate))
        df = df.drop(panel)
        count += 1
    else:
        l.append(fillRate)

after = df.shape[0]
print("{} panels removed ({:,.0f} records) due to low fill rate".\
      format(count, (before-after)))
print("{}Remaining panels fill rate is {:.1%}".format("\n", sum(l)/len(l)))

23 panels removed (140,148 records) due to low fill rate

Remaining panels fill rate is 96.8%


##### Generate new features

In [87]:
# This is mapping each hour in the day to an 8-hour block
conditions = [
    (df['hour'] > 19) | (df['hour'] < 4),
    (df['hour'] > 3) & (df['hour'] < 12)]

blocks = ['evening', 'morning']

df['block']   = np.select(conditions, blocks, default='afternoon')
df["dow"]     = df['date'].dt.dayofweek
df["weekNum"] = df['date'].dt.week
df["month"]   = df["date"].dt.month

assert (df["block"].value_counts().sum()   == df.shape[0])
assert (df["dow"].value_counts().sum()     == df.shape[0])
assert (df["weekNum"].value_counts().sum() == df.shape[0])
assert (df["month"].value_counts().sum()   == df.shape[0])

In [ ]:
output = open(datalib+"Wins and Losses.txt", "w")

In [ ]:
def report(idx, indicator):
    rec = "|".join(idx)
    rec = rec +"|"+ indicator +"\n"
    output.write(rec)

In [ ]:
# "Win" is when you have a recent customer so not enough data to forecast
# "Loss" is when you don't have recent data, so no longer active/relevant
def removeWinsandLosses(df, cutoff):
    sort = ["Country", "Program", "Customer", "Driver", "Month"]
    df = df.sort_values(sort)
    df = df.set_index(["Country", "Program", "Customer", "Driver"], drop=True)
    cutoff = datetime.datetime.strptime(cutoff, "%Y-%m-%d")
    loss = 0
    win  = 0

    for idx, val in df.groupby(level=[0, 1, 2, 3]):
        dates = val["Month"]
        if dates.iloc[-1] < cutoff:    # Check for Loss
            report(idx, "L")
            df = df.drop(idx)
            loss += 1
        else:                        # Check for Win
            if len(dates) < 12:
                win += 1
                report(idx, "W")
                df = df.drop(idx)
    print("{:,.0f} Loss and {:,.0f} wins".format(loss, win))
    return df.reset_index()

In [ ]:
df = df.reset_index()
df = removeWinsandLosses(df, "2018-01-30")

output.close()

In [ ]:
df.to_csv(datalib+"Processed.csv", sep=",", index=False)